# Introducing the NONCOV Toolbox in Python

## Introduction

## Structure

## Load NONCOVToolbox Library

In [7]:
import sys
import os

path_noncov = os.path.abspath(os.path.join('..', 'src'))

if path_noncov not in sys.path:
    sys.path.append(path_noncov)

from noncov import NONCOVToolbox

noncov = NONCOVToolbox()



          #################################################
          | --------------------------------------------- |
          |         (NC)^2I.py: NMR Calculations          |
          |         for Noncovalent Interactions          |
          | --------------------------------------------- |
          |           Introducing: NONCOVToolbox          |
          |                       -                       |
          |     A collection of functions for working     |
          |         with calculated NMR parameters        |
          |                                               |
          |               Ettore Bartalucci               |
          |     Max Planck Institute CEC & RWTH Aachen    |
          |            Worringerweg 2, Germany            |
          |                                               |
          #################################################

Stable version: 0.0.1


Working python version:
3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:

In [5]:
configdir = os.getcwd()
configdir = os.path.abspath(os.path.join('..', 'config'))
configs = os.path.join(configdir, 'configuration.yml')
print(f'Change with care, configuration file is in:', configs)

print('And looks like this...\n')
with open(configs,'r') as f:
    config_file = f.read()
    print(config_file)

Change with care, configuration file is in: d:\PhD\Data\DFT\NONCOV\DFT_simulations\codes\config\configuration.yml
And looks like this...

STRUCTURE =
SCRATCH =



In [6]:
current_dir = os.getcwd()
print(f'Current working directory is: {current_dir}')

Current working directory is: d:\PhD\Data\DFT\NONCOV\DFT_simulations\codes\results


## Modules: OrcaAnalysis

## Modules: NMRFunctions

## Modules: MolecularGraph

## Modules: DistanceScanner & RotationScanner

## Preliminary results